## 5)age group in India that has the highest percentage of people speaking                            three languages or more

In [1]:
#files used-- c18,c14(population by age)

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [3]:
#ASSIGNING COL NAMES
column_names = ['State', 'District', 'Name', 'TRU','Age-group','Persons.2','Males.2','Females.2','Persons.3','Males.3','Females.3']
#READING DATA
df = pd.read_excel('DDW-C18-0000.xlsx',names=column_names,skiprows=5)
# df = pd.read_excel('DDW-C18-0000.xlsx',skiprows=1)

In [4]:
df.columns

Index(['State', 'District', 'Name', 'TRU', 'Age-group', 'Persons.2', 'Males.2',
       'Females.2', 'Persons.3', 'Males.3', 'Females.3'],
      dtype='object')

In [5]:
df['Age-group'].unique()

array(['Total', '5-9', '10-14', '15-19', '20-24', '25-29', '30-49',
       '50-69', '70+', 'Age not stated'], dtype=object)

In [6]:
#TAKING ONLY REQUIRED COLUMNS
df = df[['State','Name','TRU','Age-group','Persons.3']]
# REMOVING URBAN AND RURAL ROWS AND AGE
idx = df.loc[(df['TRU']!='Total') | (df['Age-group'] == 'Total')].index
df = df.drop(idx)

In [7]:
df['Age-group'] = df['Age-group'].str.replace('5-9','05-09')   #BECAUSE IT IS USEFUL WHILE SORTING DATA

PREPROCESSING C14 file (POPULATION BY AGE)

In [8]:
#ASSIGNING COL NAMES
column_names = ['table_name', 'State','District', 'Name','Age-group', 'Persons', 'Males', 'Females','Persons.1', 'Males.1', 'Females.1', 'Persons.2', 'Males.2', 'Females.2']
#READING DATA
age_pop = pd.read_excel('DDW-0000C-14.xls',names=column_names,skiprows=6)


In [9]:
age_pop['Age-group'].unique()

array(['All ages', '0-4', '5-9', '10-14', '15-19', '20-24', '25-29',
       '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64',
       '65-69', '70-74', '75-79', '80+', 'Age not stated'], dtype=object)

In [10]:
# REMOVING AGE_GROUP "0-4" as not needed for analysis
ix = age_pop.loc[(age_pop['Age-group'] == 'All ages') | (age_pop['Age-group'] == '0-4')].index
age_pop = age_pop.drop(ix)

In [11]:
#MAKING SOME SPELLING CHANGES
age_pop['Name'] = age_pop['Name'].str.replace('State - ','')
age_pop['Name'] = age_pop['Name'].str.replace('India','INDIA')

In [12]:
age_pop['Age-group'] = age_pop['Age-group'].str.replace('5-9','05-09')       #BECAUSE IT IS USEFUL WHILE SORTING DATA

In [13]:
age_pop.reset_index(drop=True,inplace=True)

In [14]:
# age_pop.shape

In [15]:
#TAKING ONLY REQUIRED COLUMNS
age_pop = age_pop[['Name', 'Age-group', 'Persons']]

In [16]:
age_pop[age_pop['Name'] == 'INDIA']['Persons'].values

array([126928126, 132709212, 120526449, 111424222, 101413965,  88594951,
        85140684,  72438112,  62318327,  49069254,  39146055,  37663707,
        26454983,  19208842,   9232503,  11289005,   4489802], dtype=int64)

In [17]:
# age_pop.head()

### CONVERITING ROWS INTO CLOUMNS OF AGE GROUPS to ease calculations

In [18]:
t = age_pop.pivot_table(index=['Name'],columns='Age-group')

In [19]:
t =t.droplevel(0,axis=1)   #DROOPING MULTIINDEX FOR COLUMNS

In [20]:
t.reset_index(inplace=True)

In [21]:
# t.head()

In [22]:
t.iloc[[0],[14,15,16,17]]

Age-group,70-74,75-79,80+,Age not stated
0,4098,2251,2473,368


In [23]:
#COMBINING AGE GROUPS AS REQUIRED FOR C-19 AGE GROUP FILE
t['30-49'] = t.iloc[:,6] + t.iloc[:,7] +t.iloc[:,8] +t.iloc[:,9]
t['50-69'] = t.iloc[:,10] + t.iloc[:,11] +t.iloc[:,12] +t.iloc[:,13]
t['70+'] = t.iloc[:,14] + t.iloc[:,15] +t.iloc[:,16]

In [24]:
#TAKING ONLY REQUIRED FORMAT AGEGROUPS
t = t[['Name', '05-09', '10-14', '15-19', '20-24', '25-29','30-49', '50-69', '70+','Age not stated']]

In [25]:
# t.head()

In [26]:
#CONVERTING COLUMNS INTO ROWS (AGE-GROUPS)
t=pd.melt(t,id_vars=['Name'])

In [27]:
t = t.groupby(['Name','Age-group']).first()
t.reset_index(inplace=True)

In [28]:
t           # WE HAVE POPULATION FOR EACH GROUP AS REQUIRED FOR C18 FILE

,Name,Age-group,value
0,ANDAMAN & NICOBAR ISLANDS (35),05-09,30454
1,ANDAMAN & NICOBAR ISLANDS (35),10-14,33467
2,ANDAMAN & NICOBAR ISLANDS (35),15-19,33026
3,ANDAMAN & NICOBAR ISLANDS (35),20-24,37115
4,ANDAMAN & NICOBAR ISLANDS (35),25-29,41722
...,...,...,...
319,WEST BENGAL (19),25-29,7997909
320,WEST BENGAL (19),30-49,25343015
321,WEST BENGAL (19),50-69,12254977
322,WEST BENGAL (19),70+,3013935


In [29]:
t.shape,df.shape

((324, 3), (324, 5))

## FINDING PERCENTAGES OF EACH AGE GROUP

In [30]:
df = df.sort_values(['Name','Age-group'])    # TO MATCH WITH POPULATION DATAFRAME

In [31]:
df.reset_index(inplace=True,drop=True)

In [32]:
# df.head(10)

In [33]:
# t.head()

In [34]:
df['population'] = t['value']    #ADDING POPULATION COLUMN AS A COLUMN

In [35]:
df['percentage'] = df['Persons.3'] / t['value']   #CALCULATING RATIO

In [36]:
df.shape

(324, 7)

In [37]:
#finding indexes of maxratio
indexes = []
for i in range(36):         #iterating for states
    t = i*9                 
    indexes.append(df.iloc[t:t+9,-1].idxmax())      #finding index of maxratio

In [38]:
#MAXRATIO ROWS ARE STORED INTO A DATAFRAME
finaldf = df.iloc[indexes]

In [39]:
#TAKING ONLY REQUIRED COLUMNS AND RENAMING
resultdf = finaldf[['State','Age-group','percentage']]
resultdf.rename(columns={'State':'state/ut','Age-group':'age-group'})
resultdf.reset_index(inplace=True,drop=True)

In [40]:
#SORTING OUTPUT FILE ON STATEID
resultdf.sort_values(['State'],inplace = True)
resultdf.reset_index(drop=True,inplace=True)

In [41]:
#WRITING TO CSV
resultdf.to_csv('age-india.csv',index=False)